In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import re

from sklearn.decomposition import PCA, SparsePCA,TruncatedSVD
from sklearn import preprocessing
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import silhouette_samples, silhouette_score
from pymongo import MongoClient
import re

from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster


%matplotlib inline

In [5]:
df_c = pd.read_csv('crime_pca_data.csv')

In [4]:
cd Dimension\ Reduction

/Users/evansadler/Desktop/Project Files/Dimension Reduction


In [6]:
df_z = pd.read_csv('zillow_pca_data.csv')

In [8]:
df_f = df_c.merge(df_z, on='hood')

In [18]:
city = 'Seattle'
state = 'WA'
client = MongoClient()
db = client['neighborhood_recommender']
collection = db['hood_data' + '_' + city + '_' + state]

hood_zillow = [hood for hood in collection.find()]
df_zo = pd.DataFrame(hood_zillow, dtype = np.float64)
names = df_zo['name'].values
df_zo = df_zo[['id', 'name']]
df_zo.columns = ['id', 'hood']

In [23]:
df_f = df_f.merge(df_zo, on='hood')

In [29]:
from combine_data import CombineData

In [ ]:
cb = CombineData('Seattle', 'WA')
cb.load_data_and_engineer_features()

In [34]:
df_y = pd.read_csv('yelp_pca_data.csv')

In [40]:
df_f = df_f.merge(df_y, on = 'id')

,hood,crime,serious crime,prostitution,family vs single,expensiveness,Age,id,active,arts,nightlife,restaurants,shopping
0,Admiral,0.039380,-0.021932,-0.051377,3.874844e-02,0.023296,6.625703e-02,344019,48,16,14,27,93
1,Arbor Heights,0.058354,-0.021932,-0.069827,1.341016e-01,-0.103825,6.583657e-02,344024,23,8,4,0,75
2,Atlantic,0.006350,-0.013786,-0.016942,3.548609e-02,-0.021187,1.935341e-02,271804,36,12,6,25,102
3,Ballard,-0.094129,-0.021932,0.087794,-1.096564e-01,-0.010820,4.301891e-02,250017,79,41,59,149,199
4,Beacon Hill,-0.080443,0.009634,0.519038,1.133733e-01,-0.115577,6.685077e-02,250050,75,47,32,179,170
5,Belltown,-0.129944,-0.010731,0.043421,-2.104924e-01,0.039857,-8.434029e-02,271808,56,50,55,169,179
6,Bitter Lake,0.026877,-0.004622,-0.035219,-4.531733e-02,-0.047610,2.216094e-01,271814,30,10,6,22,97
7,Blue Ridge,0.067760,-0.021932,-0.104104,6.129286e-02,0.056672,6.718152e-02,344006,22,8,4,0,77
8,Brighton,0.008237,-0.015823,0.068669,1.097882e-01,-0.140686,2.382214e-02,250146,18,9,2,26,72
9,Broadmoor,0.073358,-0.021932,-0.109202,4.645411e-02,0.114902,2.036894e-01,250149,29,13,4,5,86


In [46]:
w = WalkscoreFeatures('Seattle', 'WA')
df_ws = w.engineer_features()

In [44]:
from walkscore_features import WalkscoreFeatures

In [61]:
df_f = df_ws.merge(df_f, on ='id')

In [79]:
names = df_f['hood']
df_f.drop('id', axis = 1, inplace = True)

In [65]:
from pca_class import ReduceFeatures

In [88]:
rf = ReduceFeatures(df_f, names)
rf.preprocessing(0)
#rf.hcluster_cols(0.3)
#rf.preprocessing(0)
df_c = rf.sparse_pca(n_components = 5, alpha = 1)
rf.best_cluster()

For n_clusters = 3 The average silhouette_score is : 0.285476657747
For n_clusters = 4 The average silhouette_score is : 0.270739924881
For n_clusters = 5 The average silhouette_score is : 0.286060148861
For n_clusters = 6 The average silhouette_score is : 0.280979887418
For n_clusters = 7 The average silhouette_score is : 0.271517841639
For n_clusters = 8 The average silhouette_score is : 0.259430615592
For n_clusters = 9 The average silhouette_score is : 0.283117971254
For n_clusters = 10 The average silhouette_score is : 0.269825435618


In [92]:
df_c

,1,2,3
transit_score,-6.590399,0.000000,0.000000
walkscore,-6.231634,0.000000,0.283360
crime,5.893753,0.000000,0.000000
serious crime,0.000000,-2.542314,-3.561765
prostitution,-3.870505,-3.160507,0.000000
family vs single,6.271583,0.000000,0.000000
expensiveness,0.000000,6.818006,0.000000
Age,0.000000,0.000000,-6.842315


In [94]:
df_f.to_csv('final_1.csv', index = False)